# Van der Pol equation - Simulation and plotting

In [ ]:
%matplotlib notebook
import math
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate
import h5py

import torch
import torch.nn as nn
import torch.jit
from torchdiffeq import odeint as odeint

#from model.DEM import DeepEuler, AdaptiveDeepEuler
#from model.Euler import Euler
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"],
    "font.size": 15.0})

In [ ]:
mu = 1.5
def vdp( t, x):
    y = np.empty(x.shape)
    y[0] = -mu*(x[1]*x[1]-1)*x[0]-x[1]
    y[1] = x[0]
    return y

In [ ]:
class NeuralODE(nn.Module):
    '''The neural ODE'''

    def __init__(self, num_in_features, num_out_features):
        super(NeuralODE, self).__init__()
        self.act    = nn.ELU()
        
        self.l_in   = nn.Linear(
            in_features = num_in_features,
            out_features= 50
        )
        self.l1   = nn.Linear(
            in_features = 50,
            out_features= 100
        )
        self.l2   = nn.Linear(
            in_features = 100,
            out_features= 50
        )
        self.l_out   = nn.Linear(
            in_features = 50,
            out_features= num_out_features
        )

    def forward(self, t, y):
        x = self.act(self.l_in(y))
        x = self.act(self.l1(x));
        x = self.act(self.l2(x));
        return self.l_out(x)

## Integrate the ODE

In [ ]:
sol = scipy.integrate.solve_ivp(vdp, [0, 500], [ 1.0, 1.0], rtol=1e-10, atol=1e-10)

You need a trained Neural ODE to run the following section. Rewrite the path given to `torch.load` to point to your model. Note that this should be a pytorch model checkpoint (not a traced model).

In [ ]:
#select device
device = torch.device('cuda:' + str(gpu) if torch.cuda.is_available() else 'cpu')

#initial condition
true_y0 = torch.tensor([[4., 3.]]).to(device)

NODE = NeuralODE(2,2)
loaded = torch.load("training/model_vdp_43_i73203_2210242102.pt")
NODE.load_state_dict(loaded['model_state_dict'])
NODE.eval()

t = torch.linspace(0,35,100)
with torch.no_grad():
    node_sol = odeint(NODE, true_y0, t, atol=1e-6, rtol=1e-6).detach().numpy()

In [ ]:
fig = plt.figure(num="NeuralODE")
ax_traj = fig.add_subplot(111)
ax_traj.cla()
ax_traj.set_xlabel('$t$')
ax_traj.set_ylabel('$x_1$')
ax_traj.plot(sol.t, sol.y[0, :], '-', color="black", label="DOPRI")
ax_traj.plot(t.cpu().numpy(), node_sol[:, 0, 0], '--', color="orange", label="NODE")
ax_traj.set_xlim(0, 35)
ax_traj.set_ylim(-4, 4)
ax_traj.legend()
fig.show()
plt.savefig("VdP_NeuralODE_11.pdf")

In [ ]:
figur = plt.figure(num="Streamslice")
ax_vecfield = figur.add_subplot(111)
ax_vecfield.cla()
ax_vecfield.set_xlabel('$x_2$')
ax_vecfield.set_ylabel('$x_1$')

resolution = 41
x,y = np.mgrid[-5:5:resolution*1j, -5:5:resolution*1j]
dydt = NODE(0, torch.Tensor(np.stack([x, y], -1).reshape(resolution * resolution, 2)).to(device)).cpu().detach().numpy()
mag = np.sqrt(dydt[:, 0]**2 + dydt[:, 1]**2).reshape(-1, 1)
dydt = (dydt / mag)
dydt = dydt.reshape(resolution, resolution, 2)

ax_vecfield.streamplot(y, x, dydt[:, :, 1], dydt[:, :, 0], color="black", linewidth=1)
ax_vecfield.set_xlim(-5, 5)
ax_vecfield.set_ylim(-5, 5)

fig.tight_layout()
fig.canvas.draw()
plt.savefig("VdP_NODE_streamslice.pdf")

## Plots

In [ ]:
plt.figure(num="Comparison")
plt.plot(sol.t,sol.y[0,:], color="black", label="Dopri")
#plt.plot(euler[:,0],euler[:,1], color="silver", label="Euler")
plt.plot(dem_sol.t,dem_sol.y[0,:], color="purple", label="DEM")
#plt.plot(dem[:,0],dem[:,1], linestyle="--", dashes=(5,5), color="cyan", label="DEM C++")
plt.plot(ad_dem_sol.t,ad_dem_sol.y[0,:], linestyle="--", dashes=(5,5), color="orange", label="ADEM")
plt.xlabel("$t$")
plt.ylabel("$x_1$")
plt.xlim([480, 500])
plt.ylim([-4,4])
plt.legend()
plt.show()

In [ ]:
sol = scipy.integrate.solve_ivp(vdp, [0, 27.5], [ 1.0, 1.0], rtol=1e-10, atol=1e-10)
t = torch.linspace(0,27.5,2)
num_points = 200
min_exponent = -7
max_exponent = -2
node_errors = np.zeros(num_points)
node_tols = np.logspace( min_exponent, max_exponent, num=num_points, base=10)
i = 0;
for tol in node_tols:
    with torch.no_grad():
        pred_y = odeint(NODE, true_y0, t, atol=tol, rtol=tol ).detach().numpy()
        #print(pred_y)
        node_errors[i] = np.abs(pred_y[-1, 0, 0] - sol.y[0,-1])
        i += 1
    

In [ ]:
node_fehlberg_errors = np.zeros(num_points)
i = 0;
for tol in node_tols:
    with torch.no_grad():
        pred_y = odeint(NODE, true_y0, t, method="fehlberg2", atol=tol, rtol=tol ).detach().numpy()
        node_fehlberg_errors[i] = np.abs(pred_y[-1, 0, 0] - sol.y[0,-1])
        i += 1
    

In [ ]:
node_bosh_errors = np.zeros(num_points)
i = 0;
for tol in node_tols:
    with torch.no_grad():
        pred_y = odeint(NODE, true_y0, t, method="bosh3", atol=tol, rtol=tol ).detach().numpy()
        node_bosh_errors[i] = np.abs(pred_y[-1, 0, 0] - sol.y[0,-1])
        i += 1
    

In [ ]:
plt.figure(num="ErrorPlot")
plt.plot(node_tols, node_errors, "-", color="black", label="Dopri")
plt.plot(node_tols, node_fehlberg_errors, "-", color="orange", label="Fehlberg2")
plt.plot(node_tols, node_bosh_errors, "-", color="purple", label="Bosh3")
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Tolerance")
plt.ylabel("$|x - x_{NODE}|$")
plt.legend()
plt.show()
plt.savefig("vdp_node_error_tol_comparison.pdf")